<a href="https://colab.research.google.com/github/UN-GCPDS/python-gcpds.EEG_Tensorflow_models/blob/main/Experimental/DW_LCAM/%5B1%5D_Main_EEG_representation_Giga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Supporting modules
#-------------------------------------------------------------------------------
import numpy as np
import scipy.io as sio
import pywt
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
import cv2
import warnings
import shutil
from scipy.signal import butter, lfilter, lfilter_zi, filtfilt
from sklearn.base import  BaseEstimator, TransformerMixin
from sklearn.model_selection import StratifiedKFold,train_test_split,ShuffleSplit
warnings.filterwarnings("ignore")
#-------------------------------------------------------------------------------
!pip install mne==0.19
import mne
from mne.decoding import CSP
from mne.io import read_raw_gdf
#-------------------------------------------------------------------------------

     |████████████████████████████████| 6.4 MB 4.9 MB/s 


In [ ]:
# Definitions-------------------------------------------------------------------
def leer_GIGA_data(path_filename,ch,vt,sbj_id):
    #--- info ------------------------------------------------------------------
    # 2  ---> sample rate 
    # 7  ---> imaginary_left 
    # 8  ---> imaginary_right 
    # 11 ---> imaginary_event 
    # 14 ---> bad_trials 
    # class1: left 
    # class2: right
    #---------------------------------------------------------------------------
    raw     = sio.loadmat(path_filename)
    eeg_raw = raw['eeg']
    sfreq   = np.float(eeg_raw[0][0][2])
    id_MI   = np.where(eeg_raw[0][0][11]==1)
    id_MI   = id_MI[1]
    raw_c1  = []
    raw_c2  = []
    y_c1    = []
    y_c2    = []
    for i in range(len(id_MI)):
        l_thr = id_MI[i]-(sfreq*2-1) 
        h_thr = id_MI[i]+(sfreq*5)
        tmp_c1 = eeg_raw[0][0][7][ch,np.int(l_thr):np.int(h_thr)]
        tmp_c2 = eeg_raw[0][0][8][ch,np.int(l_thr):np.int(h_thr)]
        raw_c1.append(tmp_c1[:,np.int(vt[0]*sfreq):np.int(vt[1]*sfreq)])
        raw_c2.append(tmp_c2[:,np.int(vt[0]*sfreq):np.int(vt[1]*sfreq)])
        y_c1.append(1.0)
        y_c2.append(2.0)    
    # remove bad trials---------------------------------------------------------
    id_bad_tr_voltage_c1 = eeg_raw[0][0][14][0][0][0][0][0]
    id_bad_tr_voltage_c2 = eeg_raw[0][0][14][0][0][0][0][1]   
    id_bad_tr_mi_c1      = eeg_raw[0][0][14][0][0][1][0][0]
    id_bad_tr_mi_c2      = eeg_raw[0][0][14][0][0][1][0][1]
    ref_axis_c1          = 1
    ref_axis_c2          = 1    
    if id_bad_tr_mi_c1.shape[0]>id_bad_tr_mi_c1.shape[1]:
        id_bad_tr_mi_c1 = id_bad_tr_mi_c1.T
    if id_bad_tr_mi_c2.shape[0]>id_bad_tr_mi_c2.shape[1]:
        id_bad_tr_mi_c2 = id_bad_tr_mi_c2.T
    if id_bad_tr_voltage_c1.shape[1] == 0:
        id_bad_tr_voltage_c1 = np.reshape(id_bad_tr_voltage_c1, (id_bad_tr_voltage_c1.shape[0], id_bad_tr_mi_c1.shape[1]))
    if id_bad_tr_voltage_c2.shape[1] == 0:
        id_bad_tr_voltage_c2 = np.reshape(id_bad_tr_voltage_c2, (id_bad_tr_voltage_c2.shape[0], id_bad_tr_mi_c2.shape[1])) 
    if (id_bad_tr_voltage_c1.shape[1] > id_bad_tr_mi_c1.shape[1]):
        if id_bad_tr_mi_c1.shape[0] == 0:
            id_bad_tr_mi_c1 = np.reshape(id_bad_tr_mi_c1, (id_bad_tr_mi_c1.shape[0],id_bad_tr_voltage_c1.shape[1]))
            ref_axis_c1     = 0
    if (id_bad_tr_voltage_c2.shape[1] > id_bad_tr_mi_c2.shape[1]):
        if id_bad_tr_mi_c2.shape[0] == 0:
            id_bad_tr_mi_c2 = np.reshape(id_bad_tr_mi_c2, (id_bad_tr_mi_c2.shape[0],id_bad_tr_voltage_c2.shape[1]))
            ref_axis_c2     = 0
    if (id_bad_tr_mi_c1.shape[0] > id_bad_tr_voltage_c1.shape[0]):
        ref_axis_c1 = 0
    if (id_bad_tr_mi_c2.shape[0] > id_bad_tr_voltage_c2.shape[0]):
        ref_axis_c2 = 0
    if (id_bad_tr_voltage_c1.shape[0] > id_bad_tr_mi_c1.shape[0]):
        ref_axis_c1 = 0
    if (id_bad_tr_voltage_c2.shape[0] > id_bad_tr_mi_c2.shape[0]):
        ref_axis_c2 = 0    
    id_bad_tr_c1 = np.concatenate((id_bad_tr_voltage_c1,id_bad_tr_mi_c1),axis=ref_axis_c1)
    id_bad_tr_c1 = id_bad_tr_c1.ravel()-1
    for ele in sorted(id_bad_tr_c1, reverse = True):  
        del raw_c1[ele]
        del y_c1[ele]
    id_bad_tr_c2 = np.concatenate((id_bad_tr_voltage_c2,id_bad_tr_mi_c2),axis=ref_axis_c2)
    id_bad_tr_c2= id_bad_tr_c2.ravel()-1
    for ele in sorted(id_bad_tr_c2, reverse = True):  
        del raw_c2[ele]
        del y_c2[ele]     
    Xraw = np.array(raw_c1 + raw_c2)
    y    = np.array(y_c1 + y_c2)  
    return Xraw, y, sfreq
#-------------------------------------------------------------------------------
def bank_filter_epochsEEG(Xraw, fs, f_frec):
    nf,ff             = f_frec.shape
    epochs,channels,T = Xraw.shape
    Xraw_f            = np.zeros((epochs,channels,T,nf))
    for f in range(nf):
        lfc = f_frec[f,0]
        hfc = f_frec[f,1]
        b,a = butter_bandpass(lfc, hfc, fs)
        zi  = lfilter_zi(b, a)
        for n in range(epochs):
            for c in range(channels):
                zi              = lfilter_zi(b, a)
                Xraw_f[n,c,:,f] = lfilter(b, a, Xraw[n,c,:],zi = zi*Xraw[n,c,0])[0]
    return Xraw_f
#-------------------------------------------------------------------------------
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq  = 0.5 * fs
    low  = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
#-------------------------------------------------------------------------------
def calculo_cwt(x,fs):
    wname     = 'cmor'
    delta     = 1/fs
    coef,freq = pywt.cwt(x.T,np.arange(1,32),wname,delta)
    return coef, freq
#-------------------------------------------------------------------------------
def cwt_feat_extraction(X,fs,freq_ref):
    X_cwt = np.zeros((X.shape[0],X.shape[1],2))
    for tr in range(X.shape[0]):#loop across trials
        for ch in range(X.shape[1]):#loop across channels
            coef, freq = calculo_cwt(np.squeeze(X[tr,ch,:,0]),fs)
            coef       = np.abs(coef)
            fb_valavg  = []
            for fb in range(freq_ref.shape[0]):#loop across filter bands
                coef_mat           = coef[np.where((freq > freq_ref[fb,0]) & (freq <freq_ref[fb,1])),:]  
                coef_mat           = np.squeeze(coef_mat[0,:,:])
                X_cwt[tr,ch,fb]    = np.mean(coef_mat.flatten()) 
    return X_cwt
#-------------------------------------------------------------------------------
from sklearn.base import  BaseEstimator, TransformerMixin
class CSP_epochs_filter_extractor(TransformerMixin,BaseEstimator):
    def __init__(self, fs,f_frec=[4,30], ncomp=4,reg='empirical'):
        self.reg = reg
        self.fs = fs
        self.f_frec = f_frec
        self.ncomp = ncomp
        
    def _averagingEEG(self,X):      
        epochs,channels,T = X.shape
        Xc = np.zeros((epochs,channels,T))
        for i in range(epochs):
            Xc[i,:,:] = X[i,:,:] - np.mean(X[i,:,:])
        return Xc    
        
    def _bank_filter_epochsEEG(self,X):
        nf,ff = self.f_frec.shape
        epochs,channels,T = X.shape
        X_f = np.zeros((epochs,channels,T,nf))
        for f in range(nf):
            lfc = self.f_frec[f,0]
            hfc = self.f_frec[f,1]
            b,a = butter_bandpass(lfc, hfc, self.fs)
            X_f[:,:,:,f] = filtfilt(b,a,X,axis=2)
        return X_f    

    def _CSP_epochsEEG(self,Xraw, y,*_):
        ncomp = self.ncomp
        mne.set_log_level('WARNING')
        epochs,channels,T,nf = Xraw.shape
        Xcsp = np.zeros((epochs,self.ncomp,nf))
        self.filters  =np.zeros((self.ncomp,channels,nf))
        csp_l = []
        for f in range(nf):
            csp_l+= [CSP(n_components=ncomp, reg=self.reg, log=False,transform_into='average_power').fit(Xraw[:,:,:,f],y)]
            Xcsp[:,:,f] = csp_l[f].transform(Xraw[:,:,:,f])
            self.filters[:,:,f] = csp_l[f].filters_[:self.ncomp]
        return csp_l, Xcsp

    def fit(self,Xraw,y, *_):
        Xraw = self._averagingEEG(Xraw)
        Xraw_f = self._bank_filter_epochsEEG(Xraw)
        self.csp_l, self.Xcsp = self._CSP_epochsEEG(Xraw_f, y)
        return self    
   
    def transform(self, Xraw, *_):
        Xraw = self._averagingEEG(Xraw)
        Xraw_f = self._bank_filter_epochsEEG(Xraw)
        epochs,channels,T,nf = Xraw_f.shape
        ncomp = self.ncomp    
        result = np.zeros((epochs,ncomp,nf))   
        for f in range(nf):
            result[:,:,f] =  self.csp_l[f].transform(Xraw_f[:,:,:,f])    
        return result 
#-------------------------------------------------------------------------------
def csp_feat_extraction(Xtrain,ytrain,Xtest,fs,f_frec): 
    # Y = W.T * X
    # A*Y = X ---- A= pinv(W.T)
    XT_train = np.zeros((Xtrain.shape[0],Xtrain.shape[1],2))
    XT_test  = np.zeros((Xtest.shape[0],Xtest.shape[1],2))
    ncomp    = 6# Biclass (4-6) -- Multiclass (8-12) 
    csp_c    = CSP_epochs_filter_extractor(fs=fs,f_frec=f_frec, ncomp=ncomp)
    XT       = csp_c.fit_transform(Xtrain,ytrain)
    Filt_    = csp_c.filters
    # train/test
    for tr in range(Xtrain.shape[0]):#loop across train trials
        for fb in range(len(f_frec)):#loop across filter bands
            Xpr_tr = []
            Xpr_tr = np.dot(Filt_[:,:,fb],Xtrain[tr,:,:])
            Xfr_tr = []
            Xfr_tr = np.dot(np.linalg.pinv(Filt_[:,:,fb]),Xpr_tr) 
            XT_train[tr,:,fb] = np.mean(np.abs(Xfr_tr),axis=1)
       
    for tr in range(Xtest.shape[0]):#loop across test trials
        for fb in range(len(f_frec)):#loop across filter bands
            Xpr_ts = []
            Xpr_ts = np.dot(Filt_[:,:,fb],Xtest[tr,:,:])
            Xfr_ts = []
            Xfr_ts = np.dot(np.linalg.pinv(Filt_[:,:,fb]),Xpr_ts)
            XT_test[tr,:,fb] = np.mean(np.abs(Xfr_ts),axis=1)

    return XT_train, XT_test
#-------------------------------------------------------------------------------
def topomap_generation(types,time_inf,time_sup,id_sbj,info):
    cmap = 'gray'
    newX = 40
    newY = 40
    for itm in range(len(types)): #len(types)
        #-----------------------------------------------------------------------
        path = '/content/drive/MyDrive/Colab Notebooks/GradCam_Paper/GigaData/data/X_'+types[itm]+'_sbj_'+str(id_sbj)+'_Tw_'+str(time_inf)+'s_'+str(time_sup)+'s.pickle'  
        with open(path, 'rb') as f:
            XT_train, XT_test, y_train, y_test = pickle.load(f)
        #-----------------------------------------------------------------------    
        try:
            os.mkdir('figures/'+str(time_inf)+'s-'+str(time_sup)+'s/'+types[itm])
        except OSError:
            print('Folder exists!')
        #-----------------------------------------------------------------------    
        # train 
        X = XT_train.copy()
        #-----------------------------------------------------------------------
        try:
            os.mkdir('figures/'+str(time_inf)+'s-'+str(time_sup)+'s/'+types[itm]+'/train')
        except OSError:
            print('Folder exists!')
        #-----------------------------------------------------------------------
        X_train_reshape = np.zeros((X.shape[0],X.shape[2],int(newX),int(newY)))
        #-----------------------------------------------------------------------
        fig_mu  = plt.figure(figsize=(10,10))
        for tr in range(X.shape[0]):
            fig_mu.clear()
            image_mu  = []
            img_mu    = []
            rho_mu    = []
            rho_mu    = (X[tr,:,0]-np.min(X[tr,:,0]))/(np.max(X[tr,:,0])-np.min(X[tr,:,0]))
            mne.viz.plot_topomap(rho_mu, info, sensors=False, show=False, cmap=cmap, contours=0)
            path_mu   = 'figures/'+str(time_inf)+'s-'+str(time_sup)+'s/'+types[itm]+'/train/sbj_'+str(id_sbj)+'_tr_'+str(tr+1)+'_fb_mu.png'
            fig_mu.savefig(fname=path_mu,dpi=40,format='png',facecolor='w')
            image_mu  = cv2.imread(path_mu,0)
            img_mu    = cv2.resize(image_mu,(int(newX),int(newY)))
            X_train_reshape[tr,0,:,:] = img_mu
        #-----------------------------------------------------------------------
        fig_beta = plt.figure(figsize=(10,10))
        for tr in range(X.shape[0]):# 
            fig_beta.clear()
            image_beta = []
            img_beta   = []
            rho_beta   = []
            rho_beta   = (X[tr,:,1]-np.min(X[tr,:,1]))/(np.max(X[tr,:,1])-np.min(X[tr,:,1]))
            mne.viz.plot_topomap(rho_beta, info, sensors=False, show=False, cmap=cmap, contours=0)
            path_beta  = 'figures/'+str(time_inf)+'s-'+str(time_sup)+'s/'+types[itm]+'/train/sbj_'+str(id_sbj)+'_tr_'+str(tr+1)+'_fb_beta.png'
            fig_beta.savefig(fname=path_beta,dpi=40,format='png',facecolor='w')
            image_beta = cv2.imread(path_beta,0)
            img_beta   = cv2.resize(image_beta,(int(newX),int(newY)))
            X_train_reshape[tr,1,:,:] = img_beta
        #-----------------------------------------------------------------------        
        X = X_train_reshape.copy()
        #-----------------------------------------------------------------------
        # test
        X1 = XT_test.copy()
        #-----------------------------------------------------------------------
        try:
            os.mkdir('figures/'+str(time_inf)+'s-'+str(time_sup)+'s/'+types[itm]+'/test')
        except OSError:
            print('Folder exists!')
        #-----------------------------------------------------------------------        
        X_test_reshape = np.zeros((X1.shape[0],X1.shape[2],int(newX),int(newY)))
        #-----------------------------------------------------------------------            
        fig_mu  = plt.figure(figsize=(10,10))
        for tr in range(X1.shape[0]):
            fig_mu.clear()
            image_mu = []
            img_mu   = []
            rho_mu   = []
            rho_mu   = (X1[tr,:,0]-np.min(X1[tr,:,0]))/(np.max(X1[tr,:,0])-np.min(X1[tr,:,0]))
            mne.viz.plot_topomap(rho_mu, info, sensors=False, show=False, cmap=cmap, contours=0)
            path_mu  = 'figures/'+str(time_inf)+'s-'+str(time_sup)+'s/'+types[itm]+'/test/sbj_'+str(id_sbj)+'_tr_'+str(tr+1)+'_fb_mu.png'
            fig_mu.savefig(fname=path_mu,dpi=40,format='png',facecolor='w')
            image_mu = cv2.imread(path_mu,0)
            img_mu   = cv2.resize(image_mu,(int(newX),int(newY)))
            X_test_reshape[tr,0,:,:] = img_mu
        #-----------------------------------------------------------------------       
        fig_beta = plt.figure(figsize=(10,10))
        for tr in range(X1.shape[0]):
            fig_beta.clear()
            image_beta = []
            img_beta   = []
            rho_beta   = []
            rho_beta   = (X1[tr,:,1]-np.min(X1[tr,:,1]))/(np.max(X1[tr,:,1])-np.min(X1[tr,:,1]))
            mne.viz.plot_topomap(rho_beta, info, sensors=False, show=False, cmap=cmap, contours=0)
            path_beta  = 'figures/'+str(time_inf)+'s-'+str(time_sup)+'s/'+types[itm]+'/test/sbj_'+str(id_sbj)+'_tr_'+str(tr+1)+'_fb_beta.png'
            fig_beta.savefig(fname=path_beta,dpi=40,format='png',facecolor='w')
            image_beta = cv2.imread(path_beta,0)
            img_beta   = cv2.resize(image_beta,(int(newX),int(newY)))
            X_test_reshape[tr,1,:,:] = img_beta
        #-----------------------------------------------------------------------  
        X1 = X_test_reshape.copy()        
        #-----------------------------------------------------------------------
        Xtr = X
        Xts = X1
        with open('/content/drive/MyDrive/Colab Notebooks/GradCam_Paper/GigaData/data/CWT_CSP_data_mubeta_8_30_Tw_'+str(time_inf)+'s_'+str(time_sup)+'s_subject'+str(id_sbj)+'_'+types[itm]+'_resized_10.pickle', 'wb') as f:
            pickle.dump([Xtr, Xts, y_train, y_test], f)
        #-----------------------------------------------------------------------
#-------------------------------------------------------------------------------

### CWT and CSP feature extraction

In [ ]:
,# Experiment information--------------------------------------------------------
th       = np.array([[0.5, 2.5],[1.5, 3.5],[2.5, 4.5],[3.5, 5.5],[4.5, 6.5]])
th_name  = np.array([[-1.5, 0.5],[-0.5, 1.5],[0.5, 2.5],[1.5, 3.5],[2.5, 4.5]])
freq_ref = np.array([[8,12],[12,30]])
Nsbj     = [1]
#-------------------------------------------------------------------------------
for sbj in range(len(Nsbj)):#loop across subjects
    for i in range(th_name.shape[0]):#loop across time windows #
        #-----------------------------------------------------------------------
        print('Subject - '+str(Nsbj[sbj])+' - Time window '+str(i+1)+' of '+str(th_name.shape[0]))
        #-----------------------------------------------------------------------
        # load EEG signals------------------------------------------------------
        name           = '/content/drive/MyDrive/Universidad-2020/CNN_GIGA/GIGAdata/s' + str(Nsbj[sbj])
        filename_train = name+'.mat'
        ch             = np.arange(0,64)
        vt             = [th[i,0],th[i,1]]
        Xraw,y,sfreq   = leer_GIGA_data(filename_train,ch,vt,Nsbj[sbj])
        fs             = sfreq

        #-----------------------------------------------------------------------
        # Filtering-------------------------------------------------------------
        f_frec    = np.transpose(np.array([[8],[30]]))
        Xraw_filt = bank_filter_epochsEEG(Xraw, fs, f_frec)
        #-----------------------------------------------------------------------  
        # Split in train/test subsets-------------------------------------------
        rs = ShuffleSplit(n_splits=1, train_size=0.9, test_size=0.1, random_state=0)
        for train_index, test_index in rs.split(y):
            X_train, y_train = Xraw_filt[train_index], y[train_index]
            X_test,  y_test  = Xraw_filt[test_index],  y[test_index]
        #-----------------------------------------------------------------------
        if i==0:
            with open('/content/drive/MyDrive/Colab Notebooks/GradCam_Paper/GigaData/data/idxs_train_test_'+str(Nsbj[sbj])+'.pickle', 'wb') as f:
                pickle.dump([train_index, test_index], f)
        #-----------------------------------------------------------------------
        # Compute CWT feature extraction----------------------------------------
        X_cwt_train = cwt_feat_extraction(X_train,fs,freq_ref)
        X_cwt_test  = cwt_feat_extraction(X_test,fs,freq_ref)
        #-----------------------------------------------------------------------
        # Compute CSP feature extraction----------------------------------------
        X_csp_train,X_csp_test = csp_feat_extraction(np.squeeze(X_train),y_train,np.squeeze(X_test),fs,freq_ref)
        #-----------------------------------------------------------------------
        # Save extracted features-----------------------------------------------
        with open('/content/drive/MyDrive/Colab Notebooks/GradCam_Paper/GigaData/data/X_cwt_sbj_'+str(Nsbj[sbj])+'_Tw_'+str(th_name[i,0])+'s_'+str(th_name[i,1])+'s.pickle', 'wb') as f:
            pickle.dump([X_cwt_train, X_cwt_test, y_train, y_test], f)
        with open('/content/drive/MyDrive/Colab Notebooks/GradCam_Paper/GigaData/data/X_csp_sbj_'+str(Nsbj[sbj])+'_Tw_'+str(th_name[i,0])+'s_'+str(th_name[i,1])+'s.pickle', 'wb') as f:
            pickle.dump([X_csp_train, X_csp_test, y_train, y_test], f)
        #-----------------------------------------------------------------------
print('Feature Extraction Done!!!\n')

### Topographic map montage

In [ ]:
# set EEG montage using standard 10-20 system-----------------------------------
channels_names = ['FP1','AF7','AF3','F1','F3','F5','F7','FT7','FC5','FC3','FC1','C1',
           'C3','C5','T7','TP7','CP5','CP3','CP1','P1','P3','P5','P7','P9','PO7',
           'PO3','O1','Iz','Oz','POz','Pz','CPz','FPz','FP2','AF8','AF4','AFz',
           'Fz','F2','F4','F6','F8','FT8','FC6','FC4','FC2','FCz','Cz','C2','C4',
           'C6','T8','TP8','CP6','CP4','CP2','P2','P4','P6','P8','P10','PO8',
           'PO4','O2']
montage = mne.channels.read_montage('standard_1020', channels_names)
info = mne.create_info(channels_names, sfreq=512, ch_types="eeg",
                         montage=montage)
f,ax = plt.subplots(1,1,figsize=(3,3))
mne.viz.plot_sensors(info, show_names=True,axes=ax)
#-------------------------------------------------------------------------------

### Topoplot generation

In [ ]:
# Load feat data----------------------------------------------------------------
th_name  = np.array([[-1.5, 0.5],[-0.5, 1.5],[0.5, 2.5],[1.5, 3.5],[2.5, 4.5]])
types    = ['cwt','csp']
Nsbj     = [30]
#-------------------------------------------------------------------------------
for sbj in range(len(Nsbj)):#loop across subjects
    try:
        os.mkdir('figures')
    except OSError:
        print('Folder exists!')
    for i in range(th_name.shape[0]):#loop across time windows
        print('Subject - '+str(Nsbj[sbj])+' - Time window '+str(i+1)+' of '+str(th_name.shape[0]))
        try:
            os.mkdir('figures/'+str(th_name[i,0])+'s-'+str(th_name[i,1])+'s')
        except OSError:
            print('Folder exists!')    
        topomap_generation(types,th_name[i,0],th_name[i,1],Nsbj[sbj],info)
    shutil.rmtree('figures', ignore_errors=True)
print('Topoplot generation Done!!!\n')
#-------------------------------------------------------------------------------